# Hometask

1) Find text to train (any book)<br>
2) Build train and validation set <br>
3) Train bidirectional language model that predicts the POS of word being based on its `n_context= 3` neighbours from the left and `n_context= 3` neighbours from the right <br>
4) Evaluate the model

In [8]:
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dense
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense

## Loading data

In [9]:
file_path = 'Dataset.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()
    
print(text[:5000])

A Preliminary Note
There is an old saying, ‘Murder will out.’ I am really unable to see why this should be so. At any rate, it is a statement impossible of proof, and one which must always remain a matter of opinion. Because certain clumsy criminals have placed themselves in full view of that dull dog, the Law, we are asked to believe that crime is invariably awkward. The logic is not very obvious. I am convinced that many a delightful member of society has found it necessary at some time or other to remove a human obstacle, and has done so undetected and undisturbed by those pangs of conscience which Society, afraid of itself, would have us believe wait upon the sinner.

Israel Rank.

Chapter I
It was the close of a bleak, autumnal afternoon. All day long in the chill and windy atmosphere the dust had been driven helter-skelter along the shabbier streets of Clapham, whirling with it the leaves which had fallen from the depressed trees in the gardens of the innumerable semi-detached vi

## Tokenization

In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

# Vectorization
sequences = tokenizer.texts_to_sequences([text])[0]
total_words = len(sequences)
print(total_words)
print(sequences)

118343
[5, 2832, 876, 43, 30, 36, 163, 471, 2833, 115, 54, 649, 2, 98, 329, 506, 3, 85, 314, 49, 51, 25, 38, 22, 57, 392, 11, 30, 5, 1548, 383, 4, 877, 6, 45, 32, 84, 99, 612, 5, 155, 4, 827, 263, 209, 2349, 1997, 24, 522, 364, 8, 237, 423, 4, 9, 828, 1754, 1, 384, 55, 63, 144, 3, 168, 9, 650, 30, 947, 777, 1, 3599, 30, 16, 41, 651, 2, 98, 686, 9, 210, 5, 1021, 687, 4, 318, 107, 132, 11, 424, 22, 79, 56, 52, 86, 3, 878, 5, 472, 2834, 6, 107, 178, 38, 1998, 6, 2350, 35, 181, 3600, 4, 778, 32, 318, 240, 4, 393, 28, 24, 139, 168, 1171, 200, 1, 3601, 206, 241, 425, 2, 11, 7, 1, 948, 4, 5, 5073, 5074, 330, 44, 125, 169, 8, 1, 3602, 6, 5075, 879, 1, 3603, 10, 29, 1392, 5076, 5077, 473, 1, 3604, 1549, 4, 542, 5078, 17, 11, 1, 3605, 32, 10, 1269, 39, 1, 3606, 1999, 8, 1, 2835, 4, 1, 2351, 1393, 2000, 5079, 352, 6, 43, 5080, 4, 2352, 728, 5081, 3607, 473, 1, 3608, 15, 1, 2001, 3609, 829, 70, 5082, 9, 1, 1172, 10, 5083, 5084, 20, 5, 173, 1270, 8, 3610, 779, 102, 3611, 1173, 5085, 36, 3612, 2353,

## Data creation

In [11]:
contexts = []
targets = []

n_context = 3
for i in range(n_context, len(sequences) - n_context):
    context = []
    context.extend(sequences[i - n_context:i])
    context.extend(sequences[i + 1:i + n_context + 1])
    target = sequences[i]
    contexts.append(context)
    targets.append(target)

contexts = np.array(contexts)
targets = np.array(targets)

print("targets: ", targets[:10])
contexts[:10]

targets:  [  43   30   36  163  471 2833  115   54  649    2]


array([[   5, 2832,  876,   30,   36,  163],
       [2832,  876,   43,   36,  163,  471],
       [ 876,   43,   30,  163,  471, 2833],
       [  43,   30,   36,  471, 2833,  115],
       [  30,   36,  163, 2833,  115,   54],
       [  36,  163,  471,  115,   54,  649],
       [ 163,  471, 2833,   54,  649,    2],
       [ 471, 2833,  115,  649,    2,   98],
       [2833,  115,   54,    2,   98,  329],
       [ 115,   54,  649,   98,  329,  506]])

## Separation of training and test data

In [12]:
X_train, X_test, y_train, y_test = train_test_split(contexts, targets, test_size=0.2)

## Creating model

In [13]:
embedding_dim = 25

model = Sequential()
model.add(Embedding(input_dim=total_words,output_dim=embedding_dim, input_length=n_context*2))
model.add(Bidirectional(LSTM(128), input_shape=(n_context * 2, 1)))
model.add(Dense(total_words, activation='softmax'))

# Compiling and training the model
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer, metrics=['accuracy'])

## Training ans evaluate model

In [14]:
history = model.fit(X_train, y_train, batch_size=100, epochs=40, validation_data=(X_test, y_test))

Epoch 1/40
947/947 [==============================] - 391s 410ms/step - loss: 6.8851 - accuracy: 0.0782 - val_loss: 6.3142 - val_accuracy: 0.1165
Epoch 2/40
947/947 [==============================] - 396s 418ms/step - loss: 5.9701 - accuracy: 0.1586 - val_loss: 5.8350 - val_accuracy: 0.1870
Epoch 3/40
947/947 [==============================] - 394s 416ms/step - loss: 5.5038 - accuracy: 0.2076 - val_loss: 5.6792 - val_accuracy: 0.2024
Epoch 4/40
947/947 [==============================] - 413s 436ms/step - loss: 5.1565 - accuracy: 0.2390 - val_loss: 5.5658 - val_accuracy: 0.2143
Epoch 5/40
947/947 [==============================] - 437s 461ms/step - loss: 4.8786 - accuracy: 0.2634 - val_loss: 5.5896 - val_accuracy: 0.2181
Epoch 6/40
947/947 [==============================] - 437s 462ms/step - loss: 4.6430 - accuracy: 0.2822 - val_loss: 5.6448 - val_accuracy: 0.2211
Epoch 7/40
947/947 [==============================] - 423s 447ms/step - loss: 4.4410 - accuracy: 0.2996 - val_loss: 5.7194 -